In [3]:
from TSPLIPLoader import TPSLIBLoader

burma14 = TPSLIBLoader('burma14')

burma14.nodes
best_cost = burma14.get_solution

In [4]:
from algorithms.HSFFPKO import HSFFPKO, universal_tsp_wrapper
from algorithms.EHO import elephant_herd_optimization
from algorithms.ACO import ant_colony
from algorithms.SA import simulated_annealing
from algorithms.SAI import simulated_annealing_improved
from algorithms.GWO import GreyWolfTSP
from algorithms.PSO import hybrid_pso_tsp
from algorithms.GA import genetic_alg 

num_nodes = len(burma14.nodes)
graph = burma14.to_graph()
matrix = burma14.to_matrix()

In [ ]:
import time 
import numpy as np

data = {
    'HSFFPKO': [],
    'EHO': [],
    'ACO': [],
    'SA' : [],
    'SAI': [],
    'GWO': [],
    'PSO': [],
    'GA': []
}

N = 10
for _ in range(N):
    start = time.perf_counter()
    output = HSFFPKO(
        Popsize=50,
        Maxiteration=500,
        LB=np.zeros(num_nodes),
        UB=np.ones(num_nodes) * 20,
        Dim=num_nodes,
        Fobj=universal_tsp_wrapper(matrix),  # MATRIX
        scouts_rate=0.15,
        num_flocks=8
    )[0] 
    end = time.perf_counter()
    data['HSFFPKO'].append([output,end - start])
    
    start = time.perf_counter()
    output = elephant_herd_optimization(graph)[1]
    end = time.perf_counter()
    data['EHO'].append([output,end - start])

    start = time.perf_counter()
    output = ant_colony(graph)[1]
    end = time.perf_counter()
    data['ACO'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing(graph, temperature=1000, alpha=0.995, iterations=1000)[1]
    end = time.perf_counter()
    data['SA'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing_improved(graph, iter_per_temp=100)[0]
    end = time.perf_counter()
    data['SAI'].append([output,end - start])

    gwo = GreyWolfTSP(graph)
    start = time.perf_counter()
    output = gwo.optimize()[1]
    end = time.perf_counter()
    data['GWO'].append([output,end - start])

    start = time.perf_counter()
    output = hybrid_pso_tsp(graph)[1]
    end = time.perf_counter()
    data['PSO'].append([output,end - start])

    start = time.perf_counter()
    output = genetic_alg(graph, pop_size=100, mutation_rate=0.01, generations=50)[1]
    end = time.perf_counter()
    data['GA'].append([output,end - start])




In [6]:
import pandas as pd

cost_dict = {k: [x[0] for x in v] for k, v in data.items()}
df_cost = pd.DataFrame(cost_dict)

runtime_dict = {k: [x[1] for x in v] for k, v in data.items()}
df_runtime = pd.DataFrame(runtime_dict)

# Get the relative error
def relative_error(costs, best_cost):
    return (costs - best_cost) / best_cost

rel_error_dict = {k: relative_error(np.array(v), best_cost) for k, v in cost_dict.items()}
df_rel_error = pd.DataFrame(rel_error_dict)

cols = ['HSFFPKO', 'EHO', 'ACO', 'SA', 'SAI', 'GWO', 'PSO', 'GA']

df = pd.DataFrame(
    np.vstack([df_cost.mean(), df_runtime.mean(), df_cost.min(), df_rel_error.mean() * 100, df_rel_error.min() * 100]),
    columns=cols,
    index=['Avg Cost', 'Avg Runtime', 'Min Cost', 'Avg Rel Error', 'Min Rel Error']
).transpose()

df.sort_values(by='Avg Cost', inplace=True)

df

,Avg Cost,Avg Runtime,Min Cost,Avg Rel Error,Min Rel Error
GA,3323.0,3.769655,3323.0,0.000000,0.000000
SAI,3323.0,2.706721,3323.0,0.000000,0.000000
EHO,3325.3,0.229097,3323.0,0.069215,0.000000
ACO,3334.4,2.840283,3323.0,0.343063,0.000000
PSO,3336.1,3.567767,3323.0,0.394222,0.000000
HSFFPKO,3373.0,1.457997,3323.0,1.504664,0.000000
GWO,3439.1,0.559129,3323.0,3.493831,0.000000
SA,3487.2,0.007120,3336.0,4.941318,0.391213


#### Weighted Sum Model Selection

In [9]:
df[['Avg Rel Error', 'Avg Runtime']]


,Avg Rel Error,Avg Runtime
GA,0.000000,3.769655
SAI,0.000000,2.706721
EHO,0.069215,0.229097
ACO,0.343063,2.840283
PSO,0.394222,3.567767
HSFFPKO,1.504664,1.457997
GWO,3.493831,0.559129
SA,4.941318,0.007120


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['Norm Rel Error', 'Norm Runtime']] = scaler.fit_transform(df[['Avg Rel Error', 'Avg Runtime']])

w_speed = 0.7
w_accuracy = 1 - w_speed

df['Score'] = (w_accuracy * df['Norm Rel Error']) + (w_speed * df['Norm Runtime'])

df[['Score']].sort_values(by='Score')

,Score
EHO,0.045500
SA,0.300000
GWO,0.314818
HSFFPKO,0.361280
SAI,0.502247
ACO,0.547924
PSO,0.686374
GA,0.700000
